In [1]:
### Defining Environment variables pointing to user folders
from os import environ
import yaml

# load configuration file containing different training folders of different models
with open(environ.get('MODELS_COMPARE','train_conf.yaml'), 'r') as file:
    train_conf = yaml.safe_load(file)
    
environ.setdefault("TARGET_TRAIN_DATASET","j100")
target_tset = environ['TARGET_TRAIN_DATASET']
print("Chosen target dataset: ",target_tset)
models_collection = environ.get("MODELS_COLLECTION",dict())

for train_key in train_conf['training_folder'].keys():
    models_collection.setdefault(train_key,dict())
    
    models_collection[train_key].setdefault('HOME_DIR', "/workarea/local/shared/"+ environ['USERNAME'])
    models_collection[train_key].setdefault('TRAINING_DATA_FOLDER',train_conf['training_folder'][train_key])
    models_collection[train_key].setdefault('MODEL_STORAGE',models_collection[train_key]['HOME_DIR']+"/trained_models"+models_collection[train_key]['TRAINING_DATA_FOLDER'])

    feather_folder = models_collection[train_key]['HOME_DIR']+"/lb-trksim-train/notebooks/feather_folder"+models_collection[train_key]['TRAINING_DATA_FOLDER']
    models_collection[train_key].setdefault("TEST_DATA"      , feather_folder+"/acceptance-test")
    models_collection[train_key].setdefault('INPUT_MODEL',models_collection[train_key]['MODEL_STORAGE']+"/models/acceptance/saved_model.pb")

_ = environ.setdefault('NB_EXPORT',"True") # whether export notebooks

Chosen target dataset:  j100


# Comparison of different acceptance models validation
##### Tested on environment `LHCb Analysis Facility` from [landerlini/lhcbaf:v0p8](https://hub.docker.com/r/landerlini/lhcbaf)

This notebook is part of the pipeline to model the acceptance of charged particles in LHCb.
In particular, it requires:
 * the preprocessing step, defined in the [Preprocessing.ipynb](./Preprocessing.ipynb) notebook
 * the training step, defined in the [Acceptance.ipynb](./Acceptance.ipynb) notebook.


## Libraries and environment
In this notebook (and other validation notebooks) we will use GPUs to process the data and build the histograms.
In most cases, the time is dominated by the graphics functions in `matplotlib` to the benefit from using GPUs is marginal, but this may change in the future when processing larger datasets.

To process data with the GPU we are using [`cupy`](https://cupy.dev/) implementing a numpy-compatible library of numerical functions accelerated by a GPU, and [cudf](https://docs.rapids.ai/api/cudf/stable/) which is interfaced with [dask](https://docs.dask.org/en/stable/dataframe.html), enabling streaming data to the GPU while processing them with `cupy`. `cudf` is strictly needed only when the GPU memory is insufficient to cope with the whole dataset to be processed as it automates the process of splitting the dataset in batches, loading each batch to GPU, apply some processing with `cupy` retrieving and store the output to free the GPU memory, and then continue with another batch. 

With the current volume of data, this is not necessary, but again, we plan to increase the training datasets soon.

> Unfortunately, `cudf` and `tensorflow` have inconsistent dependencies that, at the time of writing, make it impossible to have both of the libraries running on the GPU in the same environment. In the validation notebooks, where the most of the computing power is needed to split, sort and organize data in histograms, we will evaluate the model on CPU. 


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from os import environ

## Remove annoying warnings 
environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import tensorflow as tf

Matplotlib is building the font cache; this may take a moment.


In [3]:
import dask.dataframe as ddf
import cudf
import cupy as cp

/usr/local/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/miniconda3/lib/python3.9/site-packages/cudf/utils/gpu_utils.py:62: UserWarning: Failed to dlopen libcuda.so
  warnings.warn(str(e))


## Load data
We are now loading data in *Apache feather* format using our custom `FeatherReader`. In this case we will read the **Test** dataset as a 
`dask` dataframe.
Note that the **Test** dataset was obtained from the overall dataset in the preprocessing step and was never loaded in the training notebook, so it can be considered completely independnet of both the training and validation datasets used to define weights and architecture of the neural network model.


In [4]:
from feather_io import FeatherReader

data_reader = FeatherReader(models_collection[target_tset].get("TEST_DATA", "acceptance-test"))
test_dataset = data_reader.as_dask_dataframe()

## Load the preprocessing step

To produce plots of physically-meaningful variables, we need to invert and apply the preprocessing step to the data stored on disk.
The preprocessing step was defined in the [`Preprocessing.ipynb`](./Preprocessing.ipynb) notebook and stored in the same folder as the neural network model using `pickle`. We simply reload it from there.

In [5]:
import pickle
import os.path

for train_key in models_collection.keys():
    models_collection[train_key]['model_dir'] = os.path.dirname(models_collection[train_key].get("INPUT_MODEL", "/workarea/cloud-storage/anderlinil/models/acceptance/saved_model.pb"))
# load preprocessing step of the target model only    
preprocessing_file = os.path.join(models_collection[target_tset]['model_dir'], "tX.pkl")
with open(preprocessing_file, 'rb') as f:
    models_collection[target_tset]['preprocessing_step'] = pickle.load(f)

## Load the model
The model is loaded using the keras APIs, and summarized below for completeness. 

In [6]:
from IPython.display import HTML
for train_key in models_collection.keys():
    model_dir = models_collection[train_key]['model_dir']
    display (HTML(f"Loading model from {model_dir}"))
    models_collection[train_key]['acceptance_model'] = tf.keras.models.load_model(model_dir)
#acceptance_model.summary()

## Transform data and evaluate the model

In the next block we are defining the pipeline for loading data in chunks, apply to each chunk the inverted preprocessing step, obtain the neural network response for the entries of that data chunk, and finally upload the datachunk to GPU memory.

Loading data in chunks is a task performed by the `FeatherReader` object.
Then we apply a custom function, `my_processed_batch`, to each chunk. Such a function is obtained as a specialization of a more general function named `process_batch`, by specifying the list of feature and label names and the preprocessing step.



In [9]:
from validation_utils import invert_column_transformer

def process_batch(batch, preprocessor, features, labels):
    pX = batch[features].values  
    batch[features] = invert_column_transformer(preprocessor, pX)
    
    for train_key in models_collection.keys():
        acceptance_model = models_collection[train_key]['acceptance_model']
        batch[[f'predicted_{y}_{train_key}' for y in labels]] = acceptance_model.predict(pX, batch_size=len(pX), verbose=False)
    
    return batch

from functools import partial
my_process_batch = partial(process_batch, 
                        features=data_reader.features, 
                        labels=data_reader.labels, 
                        preprocessor=models_collection[target_tset]['preprocessing_step']
                       )

## You may need to install dask-cudf
#mamba install -n base dask-cudf -y -c rapidsai -c conda-forge -c nvidia/label/cuda-11.7.0

cdf = (
    ddf
    .map_partitions(my_process_batch, test_dataset)
    .map_partitions(cudf.DataFrame.from_pandas)
)
cdf

INFO:tensorflow:Assets written to: ram://529f5bd2-9f06-48de-8656-20e18e896d46/assets
INFO:tensorflow:Assets written to: ram://f20448ec-f604-4aeb-b55a-03450fda03a2/assets


ValueError: Metadata inference failed in `from_pandas`.

You have supplied a custom function and Dask is unable to 
determine the type of output that that function returns. 

To resolve this please provide a meta= keyword.
The docstring of the Dask function you ran should have more information.

Original error is below:
------------------------
RuntimeError('CUDA error at: /usr/local/miniconda3/include/rmm/cuda_device.hpp:56: cudaErrorStubLibrary CUDA driver is a stub library')

Traceback:
---------
  File "/usr/local/miniconda3/lib/python3.9/site-packages/dask/dataframe/utils.py", line 195, in raise_on_meta_error
    yield
  File "/usr/local/miniconda3/lib/python3.9/site-packages/dask/dataframe/core.py", line 6562, in _emulate
    return func(*_extract_meta(args, True), **_extract_meta(kwargs, True))
  File "/usr/local/miniconda3/lib/python3.9/contextlib.py", line 79, in inner
    return func(*args, **kwds)
  File "/usr/local/miniconda3/lib/python3.9/site-packages/cudf/core/dataframe.

## Sanity check, repeated

To ensure everything was correct in the loading and evaluation of the model, we repeat the comparison on the distribution of labels and predictions. The distributions are qualitatively comparable.

In [ ]:
plt.figure(figsize=(15,2))
df = cdf.head(1_000_000, npartitions=-1)
plt.hist(df.acceptance, bins=np.linspace(0, 1, 101), label="In Acceptance (test sample)")
for train_key in models_collection.keys():
    plt.hist(df[f'predicted_acceptance_{train_key}'], bins=np.linspace(0, 1, 101), label=f"In Acceptance (Model {train_key})", histtype='step', linewidth=2)
plt.legend(loc="upper center")
plt.xlabel("In acceptance")
plt.ylabel("Number of particles")
plt.yscale('log')
plt.show()

## Adding derived variables to the dataset

To build the histograms in kinematic bins we need may need to add some variable.
Using `dask` and `cudf`, the computation of these new variables is *lazy*: it is delayed to the time when the result is read.
At that point, the chunks are iteratively loaded from disk and the whole sequence of operations is performed on each chunk.

In [8]:
def compute_momentum(df):
    df['mc_p'] = 10**df['mc_log10_p']
    df['mc_pz'] = (df.mc_p**2 / (1 + df.mc_tx**2 + df.mc_ty**2))**0.5
    df['mc_px'] = df.mc_pz * df.mc_tx
    df['mc_py'] = df.mc_pz * df.mc_ty
    df['mc_pt'] = (df.mc_px**2 + df.mc_py**2)**0.5
    return df

def compute_eta(df):    
    mc_theta = cp.arcsin(df.mc_pt/df.mc_p)
    df['mc_eta'] = -cp.log(cp.tan(mc_theta/2))
    return df

def compute_phi(df):
    df['mc_phi'] = cp.arctan2(df.mc_py, df.mc_px)
    return df
    

cdf = (cdf.map_partitions(compute_momentum)
        .map_partitions(compute_eta)
        .map_partitions(compute_phi)
      )

NameError: name 'cdf' is not defined

## Acceptance histograms in momentum and pseudorapidity bins

Here we report the comparison of the distribution of events:
<UL>
<LI> <SPAN style="color: blue;" > Generated (without cuts or requirements) </SPAN>
<LI> <SPAN style="color: orange;" > Selected, or "in acceptance (true)" <SPAN>
<LI> <SPAN style="color: green;" > Weighted, or "in acceptance (predicted)" </SPAN>
</UL>

In [13]:
p_boundaries = [500, 3_000, 5_000, 10_000, 30_000, 100_000, 200_000]

retain = ['acceptance', 'mc_eta', 'mc_log10_p'] + [f'predicted_acceptance_{train_key}' for train_key in models_collection]

for part in ['h', 'mu', 'e']:
    part_df = cdf.query(f"mc_is_{part} == 1 and mc_eta > 0 and mc_eta < 12")[retain].compute()
    
    plt.figure(figsize=(20,3))
    for iPlot, (p_min, p_max) in enumerate(zip(p_boundaries[:-1], p_boundaries[1:]), 1):
        ax = plt.subplot(1, len(p_boundaries)-1, iPlot)
        bin_df = part_df.query(f"mc_log10_p > {np.log10(p_min)} and mc_log10_p < {np.log10(p_max)}")
        denominator, bins = cp.histogram(bin_df.mc_eta.values, bins=np.linspace(0, 10, 101))
        true_numerator, _ = cp.histogram(bin_df.query('acceptance==1').mc_eta.values, bins=bins)
        pred_numerator = dict()
        for train_key in models_collection.keys():
            pred_numerator[train_key],_ = cp.histogram(bin_df.mc_eta.values, weights=bin_df[f'predicted_acceptance_{train_key}'].values, bins=bins)
    
        bins = bins.get()
        x = (bins[:-1] + bins[1:])/2
        plt.hist(x, bins=bins, weights=denominator.get(), histtype='step', linewidth=2, alpha=0.5, label="Generated")
        for train_key in models_collection.keys():
            plt.hist(x, bins=bins, weights=pred_numerator[train_key].get(), histtype='step', linewidth=2, label=f"In acceptance (model {train_key})")
        plt.hist(x, bins=bins, weights=true_numerator.get(), histtype='step', linewidth=2, label="In acceptance (true)")
        plt.xlim(bins[0], bins[-1])
        
        part_name = dict(h="Hadrons", mu='Muons', e='Electrons')[part]
        plt.title(f"Acceptance for $p \in$ [{p_min/1e3:.1f}, {p_max/1e3:.1f}] GeV/$c$")
        plt.xlabel("Pseudorapidity $\eta$")
        plt.ylabel(f"Number of $\mathbf{{{part_name}}}$")
        
        handles, labels = ax.get_legend_handles_labels()        
        plt.tight_layout()
    plt.subplot(1, len(p_boundaries), iPlot+1)
    plt.legend(handles, labels)
    plt.axis('off')
    plt.show()

MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /usr/local/miniconda3/include/rmm/mr/device/cuda_memory_resource.hpp

## Acceptance histograms in momentum and polar angle bins

Here we report the comparison of the distribution of events:
<UL>
<LI> <SPAN style="color: blue;" > Generated (without cuts or requirements) </SPAN>
<LI> <SPAN style="color: orange;" > Selected, or "in acceptance (true)" <SPAN>
<LI> <SPAN style="color: green;" > Weighted, or "in acceptance (predicted)" </SPAN>
</UL>

In [ ]:
p_boundaries = [500, 3_000, 5_000, 10_000, 30_000, 100_000, 200_000]

retain = ['acceptance', 'mc_phi', 'mc_log10_p'] + [f'predicted_acceptance_{train_key}' for train_key in models_collection]

for part in ['h', 'mu', 'e']:
    part_df = cdf.query(f"mc_is_{part} == 1 and mc_phi > -3.5 and mc_phi < 3.5")[retain].compute()
    
    plt.figure(figsize=(20,3))
    for iPlot, (p_min, p_max) in enumerate(zip(p_boundaries[:-1], p_boundaries[1:]), 1):
        ax = plt.subplot(1, len(p_boundaries)-1, iPlot)
        bin_df = part_df.query(f"mc_log10_p > {np.log10(p_min)} and mc_log10_p < {np.log10(p_max)}")
        denominator, bins = cp.histogram(bin_df.mc_phi.values, bins=np.linspace(-np.pi, np.pi, 51))
        true_numerator, _ = cp.histogram(bin_df.query('acceptance==1').mc_phi.values, bins=bins)
        pred_numerator = dict()
        for train_key in models_collection.keys():
            pred_numerator[train_key],_ = cp.histogram(bin_df.mc_phi.values, weights=bin_df[f'predicted_acceptance_{train_key}'].values, bins=bins)
    
        bins = bins.get()
        x = (bins[:-1] + bins[1:])/2
        plt.hist(x, bins=bins, weights=denominator.get(), histtype='step', linewidth=2, alpha=0.5, label="Generated")
        for train_key in models_collection.keys():
            plt.hist(x, bins=bins, weights=pred_numerator[train_key].get(), histtype='step', linewidth=2, label=f"In acceptance (model {train_key})")
        
        plt.hist(x, bins=bins, weights=true_numerator.get(), histtype='step', linewidth=2, label="In acceptance (true)")
        plt.xlim(bins[0], bins[-1])

        part_name = dict(h="Hadrons", mu='Muons', e='Electrons')[part]
        plt.title(f"Acceptance for $p \in$ [{p_min/1e3:.1f}, {p_max/1e3:.1f}] GeV/$c$")
        plt.xlabel("Azimuthal angle $\phi$")
        plt.ylabel(f"Number of $\mathbf{{{part_name}}}$")
        
        handles, labels = ax.get_legend_handles_labels() 
        plt.tight_layout()
    plt.subplot(1, len(p_boundaries), iPlot+1)
    plt.legend(handles, labels)
    plt.axis('off')
    plt.show()

MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /usr/local/miniconda3/include/rmm/mr/device/cuda_memory_resource.hpp

## Metrics

In [ ]:
m = tf.keras.metrics.BinaryAccuracy()

p_boundaries = [500, 3_000, 5_000, 10_000, 30_000, 100_000, 200_000]

retain = ['acceptance', 'mc_eta', 'mc_log10_p'] + [f'predicted_acceptance_{train_key}' for train_key in models_collection]

for part in ['h', 'mu', 'e']:
    part_df = cdf.query(f"mc_is_{part} == 1 and mc_eta > 0 and mc_eta < 12")[retain].compute()
    metrics = dict(
        accuracy=   {train_key:[] for train_key in models_collection.keys()},
        precision=  {train_key:[] for train_key in models_collection.keys()},
        specificity={train_key:[] for train_key in models_collection.keys()}
    )
    
    fig,ax = plt.subplots(1,1,figsize=(20,3))
    for iPlot, (p_min, p_max) in enumerate(zip(p_boundaries[:-1], p_boundaries[1:]), 1):
        
        bin_df = part_df.query(f"mc_log10_p > {np.log10(p_min)} and mc_log10_p < {np.log10(p_max)}")
        for train_key in models_collection.keys():
            m.update_state(bin_df['acceptance'].values,bin_df[f'predicted_acceptance_{train_key}'].values)
            metrics['accuracy'][train_key].append(m.result().numpy())
    
    x    = (p_boundaries[1:] + p_boundaries[:-1])/2
    xerr = (p_boundaries[1:] - p_boundaries[:-1])/2
    for train_key in models_collection.keys():
        y = metrics['accuracy'][train_key]
        ax.errorbar(x=x, y=y, xerr=xerr, yerr=None, label=f"Accuracy (model {train_key})")
    ax.set_ylim(0, 1)
    ax.legend()
    part_name = dict(h="Hadrons", mu='Muons', e='Electrons')[part]
    ax.set_title(f"Metrics for $p$")
    ax.set_xlabel("GeV/$c$")
    ax.set_ylabel(f"$\mathbf{{{part_name}}}$")
    plt.tight_layout()
    plt.show()
    break

## Conclusion 

In this notebook we produced some plots showing the agreement between the acceptance obtained from detailed simulation and the modeled acceptance probability.

Additional comparisons and plots might be added in the future.

In [12]:
### export notebooks for comparisons
if environ.get('NB_EXPORT',"False")=="True":
    import os,yaml
    nb_save  = "/workarea/local/shared/scapelli/notebooks_exports"         # export output dir
    nb_save  = nb_save+models_collection[target_tset]['TRAINING_DATA_FOLDER'] # according to train data
    if not os.path.isdir(nb_save): os.makedirs(nb_save)                    # create if not exists
    nbs_path = "/workarea/local/shared/scapelli/lb-trksim-train/notebooks" # notebooks folder
    nb_filename = "Acceptance-validation-comparison.ipynb"                            # notebook name
    extensions  = ["html","pdf"]                                           # export formats
    print("Exporting models comparison YAML file")
    with open(nb_save+'/'+nb_filename.replace("ipynb","yaml"), 'w') as file:
        yaml.dump(train_conf, file)
        
    for ext in extensions:
        os.system("jupyter nbconvert --log-level=40 --no-input --output-dir {0} --to {1} {2}/{3}".format(nb_save,ext.upper(),nbs_path,nb_filename))
    print("Exported {} as {} in {}".format(nb_filename,','.join(extensions),nb_save))

Exporting models comparison YAML file
Exported Acceptance-validation-comparison.ipynb as html,pdf in /workarea/local/shared/scapelli/notebooks_exports/j100
